Test Notebook to work on Spotify songs' popularity prediction while trying to have a Green AI approach

In [1]:
#Librairies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import codecarbon
from codecarbon import EmissionsTracker
import warnings
warnings.filterwarnings('ignore')
import torch

The dataset, quickly

In [2]:
spotify_songs = pd.read_csv('/Users/camille.hascoet/Documents/Green AI/Datasets/tracks.csv')
spotify_songs.head()
#number of rows
dataset_length = len(spotify_songs)
print(dataset_length)

586672


In [3]:
#The 10 most popular songs
spotify_songs.sort_values(by='popularity', ascending=False).head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
93802,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),100,198082,1,"['Justin Bieber', 'Daniel Caesar', 'Giveon']","['1uNFoZAHBGtllmzznpCI3s', '20wkVLutqVOYrc0kxF...",2021-03-19,0.677,0.696,0,-6.181,1,0.1190,0.32100,0.000000,0.4200,0.464,90.030,4
93803,7lPN2DXiMsVn7XUKtOW1CS,drivers license,99,242014,1,['Olivia Rodrigo'],['1McMsnEElThX1knmY4oliG'],2021-01-08,0.585,0.436,10,-8.761,1,0.0601,0.72100,0.000013,0.1050,0.132,143.874,4
93804,3Ofmpyhv5UAQ70mENzB277,Astronaut In The Ocean,98,132780,0,['Masked Wolf'],['1uU7g3DNSbsu0QjSEqZtEd'],2021-01-06,0.778,0.695,4,-6.865,0,0.0913,0.17500,0.000000,0.1500,0.472,149.996,4
92810,5QO79kh1waicV47BqGRL3g,Save Your Tears,97,215627,1,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.680,0.826,0,-5.487,1,0.0309,0.02120,0.000012,0.5430,0.644,118.051,4
92811,6tDDoYIxWvMLTdKpjFkc1B,telepatía,97,160191,0,['Kali Uchis'],['1U1el3k54VvEUzo3ybLPlM'],2020-12-04,0.653,0.524,11,-9.016,0,0.0502,0.11200,0.000000,0.2030,0.553,83.970,4
93805,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,96,242096,0,"['Bruno Mars', 'Anderson .Paak', 'Silk Sonic']","['0du5cEVh5yTK9QJze8zA0C', '3jK9MiCrA42lLAdMGU...",2021-03-05,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,4
92813,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,96,200040,0,['The Weeknd'],['1Xyo4u8uXC1ZmMpatF05PJ'],2020-03-20,0.514,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4
92814,6f3Slt0GbA2bPZlz0aIFXN,The Business,95,164000,0,['Tiësto'],['2o5jDhtHVPhrJdv3cEQ99Z'],2020-09-16,0.798,0.620,8,-7.079,0,0.2320,0.41400,0.019200,0.1120,0.235,120.031,4
92816,3FAJ6O0NOHQV8Mc5Ri6ENp,Heartbreak Anniversary,94,198371,0,['Giveon'],['4fxd5Ee7UefO4CUXgwJ7IP'],2020-03-27,0.449,0.465,0,-8.964,1,0.0791,0.52400,0.000001,0.3030,0.543,89.087,3
92819,1xK1Gg9SxG8fy2Ya373oqb,Bandido,94,232853,0,"['Myke Towers', 'Juhn']","['7iK8PXO48WeuP03g8YR51W', '2LmcxBak1alK1bf7d1...",2020-12-10,0.713,0.617,8,-4.637,1,0.0887,0.12200,0.000000,0.0962,0.682,168.021,4


In [4]:
#extract the features
X = spotify_songs.drop(['popularity', 'id', 'name', 'artists', 'id_artists', 'release_date'], axis=1)
y = spotify_songs['popularity']

In [5]:
# find correlation between features and target
correlation = X.corrwith(y)
correlation = correlation.sort_values(ascending=False)
correlation

loudness            0.327028
energy              0.302315
explicit            0.211758
danceability        0.187000
time_signature      0.086759
tempo               0.071364
duration_ms         0.027681
key                 0.015299
valence             0.004643
mode               -0.033655
speechiness        -0.047357
liveness           -0.048740
instrumentalness   -0.236487
acousticness       -0.370882
dtype: float64

In [6]:
X_drop = X.drop(['mode', 'valence', 'key', 'duration_ms', 'speechiness', 'liveness'], axis=1)

input_size = X_drop.shape[1]
output_size = 1
hidden_size = 50
max_epochs = 30

In [7]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf

2024-03-22 14:25:32.691548: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(10000, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='linear')
])

model.compile(optimizer='adam', loss='mean_absolute_error')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                450       
                                                                 
 dense_1 (Dense)             (None, 10000)             510000    
                                                                 
 dense_2 (Dense)             (None, 400)               4000400   
                                                                 
 dense_3 (Dense)             (None, 400)               160400    
                                                                 
 dense_4 (Dense)             (None, 200)               80200     
                                                                 
 dense_5 (Dense)             (None, 200)               40200     
                                                                 
 dense_6 (Dense)             (None, 100)               2

In [9]:
#preprocessing features standard scaler
from sklearn.preprocessing import StandardScaler
X_scaler =StandardScaler()
X_scaled = X_scaler.fit_transform(X_drop)

y_scaled = y.values.reshape(-1,1) / 100

#shuffle and take 20000 samples
from sklearn.utils import shuffle

X_shuffled, y_shuffled = shuffle(X_scaled, y_scaled, random_state=1234)

X_train = X_shuffled[:20000]
y_train = y_shuffled[:20000]

X_test = X_shuffled[20000:25000]
y_test = y_shuffled[20000:25000]

print(y_scaled.mean(), y_train.mean())


0.2757005277224752 0.2738955


In [10]:
tracker = EmissionsTracker(log_level='error')
tracker.start()

model.fit(X_train, y_train, epochs=max_epochs, validation_data=(X_test, y_test))

total_emissions_normal = tracker.stop()

Epoch 1/30
625/625 [==============================] - 58s 54ms/step - loss: 0.1337 - val_loss: 0.1315
Epoch 2/30
625/625 [==============================] - 35s 55ms/step - loss: 0.1282 - val_loss: 0.1296
Epoch 3/30
625/625 [==============================] - 34s 54ms/step - loss: 0.1270 - val_loss: 0.1274
Epoch 4/30
625/625 [==============================] - 32s 51ms/step - loss: 0.1261 - val_loss: 0.1265
Epoch 5/30
625/625 [==============================] - 29s 46ms/step - loss: 0.1253 - val_loss: 0.1265
Epoch 6/30
625/625 [==============================] - 30s 48ms/step - loss: 0.1249 - val_loss: 0.1277
Epoch 7/30
625/625 [==============================] - 30s 48ms/step - loss: 0.1243 - val_loss: 0.1283
Epoch 8/30
625/625 [==============================] - 29s 46ms/step - loss: 0.1240 - val_loss: 0.1273
Epoch 9/30
625/625 [==============================] - 29s 46ms/step - loss: 0.1234 - val_loss: 0.1267
Epoch 10/30
625/625 [==============================] - 30s 48ms/step - loss: 0.123

ERROR: EnergyDriver_executeCommands [via readSample] returned 0xe00002bc
ERROR: EnergyDriver_executeCommands [via readSample] returned 0xe00002bc


In [11]:
print(f"Total emissions for normal model: {total_emissions_normal*100:.4f} gCO2")

Total emissions for normal model: 0.0354 gCO2


In [12]:
quantize_model = tfmot.quantization.keras.quantize_model

model_to_quantize = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(10000, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='linear')
])
model_to_quantize.summary()
q_aware_model = quantize_model(model_to_quantize)

q_aware_model.compile(optimizer='adam', loss='mean_absolute_error')

q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 50)                450       
                                                                 
 dense_9 (Dense)             (None, 10000)             510000    
                                                                 
 dense_10 (Dense)            (None, 400)               4000400   
                                                                 
 dense_11 (Dense)            (None, 400)               160400    
                                                                 
 dense_12 (Dense)            (None, 200)               80200     
                                                                 
 dense_13 (Dense)            (None, 200)               40200     
                                                                 
 dense_14 (Dense)            (None, 100)              

In [13]:
tracker.start()
q_aware_model.fit(X_train, y_train, epochs=max_epochs)
total_emissions_qaware = tracker.stop()

Epoch 1/30
625/625 [==============================] - 47s 67ms/step - loss: 0.1337
Epoch 2/30
625/625 [==============================] - 38s 61ms/step - loss: 0.1279
Epoch 3/30
625/625 [==============================] - 37s 60ms/step - loss: 0.1268
Epoch 4/30
625/625 [==============================] - 38s 61ms/step - loss: 0.1258
Epoch 5/30
625/625 [==============================] - 40s 63ms/step - loss: 0.1253
Epoch 6/30
625/625 [==============================] - 35s 56ms/step - loss: 0.1250
Epoch 7/30
625/625 [==============================] - 37s 60ms/step - loss: 0.1243
Epoch 8/30
625/625 [==============================] - 35s 56ms/step - loss: 0.1243
Epoch 9/30
625/625 [==============================] - 38s 61ms/step - loss: 0.1241
Epoch 10/30
625/625 [==============================] - 36s 57ms/step - loss: 0.1231
Epoch 11/30
625/625 [==============================] - 36s 58ms/step - loss: 0.1229
Epoch 12/30
625/625 [==============================] - 36s 58ms/step - loss: 0.1229
E

In [14]:
print(f"Total emissions for quantized aware model: {total_emissions_qaware*1000:.4f} gCO2")

Total emissions for quantized aware model: 0.7261 gCO2


In [15]:
#compare inference time and precision
import time

def inference_time_and_precision(model, X, y_test, loss):
    start = time.time()
    #model evaluate function
    loss_value = model.evaluate(X, y_test)
    end = time.time()

    inference_time = end - start
    
    return inference_time, loss_value

In [16]:
qat_inference_time, qat_loss = inference_time_and_precision(q_aware_model, X_test, y_test, tf.keras.losses.MeanAbsoluteError())

print(f"Inference time for quantized aware model: {qat_inference_time:.4f} seconds")

print(f"Loss for quantized aware model: {qat_loss:.4f}")

normal_inference_time, normal_loss = inference_time_and_precision(model, X_test, y_test, tf.keras.losses.MeanAbsoluteError())

print(f"Inference time for normal model: {normal_inference_time:.4f} seconds")

print(f"Loss for normal model: {normal_loss:.4f}")


157/157 [==============================] - 3s 12ms/step - loss: 0.1275
Inference time for quantized aware model: 3.4459 seconds
Loss for quantized aware model: 0.1275
157/157 [==============================] - 1s 6ms/step - loss: 0.1259
Inference time for normal model: 1.1066 seconds
Loss for normal model: 0.1259


In [17]:
#dataframe of the results

results = pd.DataFrame({
    'model': ['normal', 'quantized aware'],
    'emissions': [total_emissions_normal*1000, total_emissions_qaware*1000],
    'inference_time (5000 samples, sec)': [normal_inference_time, qat_inference_time],
    'loss': [normal_loss, qat_loss]
})

results

,model,emissions,"inference_time (5000 samples, sec)",loss
0,normal,0.353644,1.106617,0.125856
1,quantized aware,0.726065,3.445926,0.127549


In [18]:
#bigger model
hidden_size_1 = 1000
hidden_size_2 = 1000
hidden_size_3 = 1000
max_epochs_bigger = 10

model_bigger = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size_1, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_size_2, activation='relu'),
    tf.keras.layers.Dense(hidden_size_3, activation='relu'),
    tf.keras.layers.Dense(output_size)
])

model_bigger.compile(optimizer='adam', loss='mean_absolute_error')

model_bigger.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 1000)              9000      
                                                                 
 dense_17 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_18 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_19 (Dense)            (None, 1)                 1001      
                                                                 
Total params: 2012001 (7.68 MB)
Trainable params: 2012001 (7.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model_bigger_quantize = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_size_1, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_size_2, activation='relu'),
    tf.keras.layers.Dense(hidden_size_3, activation='relu'),
    tf.keras.layers.Dense(output_size)
])

quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model_bigger = quantize_model(model_bigger_quantize)

q_aware_model_bigger.compile(optimizer='adam', loss='mean_absolute_error')

q_aware_model_bigger.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (Quantize  (None, 8)                 3         
 Layer)                                                          
                                                                 
 quant_dense_20 (QuantizeWr  (None, 1000)              9005      
 apperV2)                                                        
                                                                 
 quant_dense_21 (QuantizeWr  (None, 1000)              1001005   
 apperV2)                                                        
                                                                 
 quant_dense_22 (QuantizeWr  (None, 1000)              1001005   
 apperV2)                                                        
                                                                 
 quant_dense_23 (QuantizeWr  (None, 1)                

In [20]:
tracker.start()
model_bigger.fit(X_train, y_train, epochs=max_epochs_bigger)
total_emissions_normal_bigger = tracker.stop()

print(f"Total emissions for normal bigger model: {total_emissions_normal_bigger*1000:.4f} gCO2")

tracker.start()
q_aware_model_bigger.fit(X_train, y_train, epochs=max_epochs_bigger)
total_emissions_qaware_bigger = tracker.stop()

print(f"Total emissions for quantized aware bigger model: {total_emissions_qaware_bigger*1000:.4f} gCO2")

qat_inference_time_bigger, qat_loss_bigger = inference_time_and_precision(q_aware_model_bigger, X_test, y_test, tf.keras.losses.MeanAbsoluteError())

print(f"Inference time for quantized aware bigger model: {qat_inference_time_bigger:.4f} seconds")

print(f"Loss for quantized aware bigger model: {qat_loss_bigger:.4f}")

normal_inference_time_bigger, normal_loss_bigger = inference_time_and_precision(model_bigger, X_test, y_test, tf.keras.losses.MeanAbsoluteError())

print(f"Inference time for normal bigger model: {normal_inference_time_bigger:.4f} seconds")

print(f"Loss for normal bigger model: {normal_loss_bigger:.4f}")

Epoch 1/10
625/625 [==============================] - 11s 14ms/step - loss: 0.1344
Epoch 2/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1270
Epoch 3/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1259
Epoch 4/10
625/625 [==============================] - 8s 12ms/step - loss: 0.1251
Epoch 5/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1249
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1246
Epoch 7/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1239
Epoch 8/10
625/625 [==============================] - 7s 12ms/step - loss: 0.1238
Epoch 9/10
625/625 [==============================] - 7s 11ms/step - loss: 0.1235
Epoch 10/10
625/625 [==============================] - 7s 12ms/step - loss: 0.1232
Total emissions for normal bigger model: 0.7546 gCO2
Epoch 1/10
625/625 [==============================] - 12s 14ms/step - loss: 0.1360
Epoch 2/10
625/625 [======================

In [21]:
bigger_results = pd.DataFrame({
    'model': ['normal bigger', 'quantized aware bigger'],
    'emissions': [total_emissions_normal_bigger*1000, total_emissions_qaware_bigger*1000],
    'inference_time (5000 samples, sec)': [normal_inference_time_bigger, qat_inference_time_bigger],
    'loss': [normal_loss_bigger, qat_loss_bigger]
})

bigger_results

,model,emissions,"inference_time (5000 samples, sec)",loss
0,normal bigger,0.754577,1.341342,0.126140
1,quantized aware bigger,0.785458,1.817299,0.127521


In [22]:
#check the size of the models
import os

def get_model_size(model):
    model.save('temp_model.h5')
    size = os.path.getsize('temp_model.h5')
    os.remove('temp_model.h5')
    return size

normal_model_size = get_model_size(model)

print(f"Normal model size: {normal_model_size} bytes")

q_aware_model_size = get_model_size(q_aware_model)

print(f"Quantized aware model size: {q_aware_model_size} bytes")

Normal model size: 57807552 bytes
Quantized aware model size: 57842896 bytes
